In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as warn
import sys
import env
import numpy as np

sys.path.append(env.util_repo)

from pydataset import data
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from scipy import stats
from utilities import evaluate_hypothesis_ttest, evaluate_hypothesis_pcorrelation

warn.filterwarnings("ignore")

### 1. Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.

### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |

In [8]:
# Dogs are the 'positive' outcome
accuracy = (46 + 34) / (46 + 7 + 13 + 34)
print(f"Accuracy:  {round(accuracy * 100, 2)}%")

precision = 46 / (46 + 13)
print(f"Precision:  {round(precision * 100, 2)}%")

recall = 46 / (46 + 7)
print(f"Recall:  {round(recall * 100, 2)}%")

Accuracy:  80.0%
Precision:  77.97%
Recall:  86.79%


<li>In the context of this problem, what is a false positive?
<p>If we set dogs as the 'positive' outcome, false positives are when we predict dog, but actually get a cat. There were 13 false positives.

<li>In the context of this problem, what is a false negative?
<p>A false negative is when we predict cat, but actually get a dog. There were 7 false negatives.

<li>How would you describe this model?
<p>This model is highly accurate since it correctly predicted the outcomes 80% of the time which means that percentage of all predictions matched the actual result. It is also highly precise when predicting positive outcomes since it has a precision of 78% which means that percentage of dog predictions were correct. The recall rate is also high at 87% which means that percentage of actual dog results was predicted correctly.

### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.

#### Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions can be found here.

#### Use the predictions dataset and pandas to help answer the following questions:

In [98]:
ducks_df = pd.read_csv(env.data_path + "/c3.csv")
ducks_df.head()

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect


In [99]:
ducks_df.shape

(200, 4)

In [100]:
model_1 = ducks_df[['actual', 'model1']]
model_1.rename(columns={'model1' : 'predict'}, inplace=True)
pd.crosstab(model_1.predict, model_1.actual)

actual,Defect,No Defect
predict,,
Defect,8,2
No Defect,8,182


In [101]:
model_2 = ducks_df[['actual', 'model2']]
model_2.rename(columns={'model2' : 'predict'}, inplace=True)
pd.crosstab(model_2.predict, model_2.actual)

actual,Defect,No Defect
predict,,
Defect,9,81
No Defect,7,103


In [102]:
model_3 = ducks_df[['actual', 'model3']]
model_3.rename(columns={'model3' : 'predict'}, inplace=True)
pd.crosstab(model_3.predict, model_3.actual)

actual,Defect,No Defect
predict,,
Defect,13,86
No Defect,3,98


<li>An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [103]:
# Define 'Defect' as the positive outcome then measure recall of each model.
# Since missing a defective duck would be costly, we should optimize for recall.
model_1_recall = 8 / (8 + 8)
model_1_recall

0.5

In [108]:
defects_model_1 = ducks_df[ducks_df.model1 == 'defect']
# See solutions for rest of alternative method

In [30]:
model_2_recall = 9 / (9 + 7)
model_2_recall

0.5625

In [31]:
model_3_recall = 13 / (13 + 3)
model_3_recall

0.8125

#### Model 3 has the best recall at 81.25% suggesting it minimizes false negatives.

<li> Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

In [32]:
# Define 'Defect' as the positive outcome and then measure precision.
# Since false positives become costly with the free vacation, it becomes important to optimize for precision.
# This will maximize the percentage of positive predictions.

model_1_precision = 8 / (8 + 2)
model_1_precision

0.8

In [33]:
model_2_precision = 9 / (9 + 81)
model_2_precision

0.1

In [35]:
model_3_precision = 13 / (13 + 86)
model_3_precision

0.13131313131313133

#### Model 1 has the highest precision meaning it has the lowest amount of false positives.

### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).

<p>At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

<p>Several models have already been developed with the data, and you can find their results here.

<p>Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

In [126]:
cats_and_dogs_df = pd.read_csv(env.data_path + "/gives_you_paws.csv")
cats_and_dogs_df.head()

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog


In [127]:
cats_and_dogs_df.shape

(5000, 5)

In [128]:
model_1 = cats_and_dogs_df[['actual', 'model1']]
model_1.rename(columns={'model1' : 'predict'}, inplace=True)
pd.crosstab(model_1.actual, model_1.predict)

predict,cat,dog
actual,,
cat,1423,323
dog,640,2614


In [129]:
model_2 = cats_and_dogs_df[['actual', 'model2']]
model_2.rename(columns={'model2' : 'predict'}, inplace=True)
pd.crosstab(model_2.actual, model_2.predict)

predict,cat,dog
actual,,
cat,1555,191
dog,1657,1597


In [130]:
model_3 = cats_and_dogs_df[['actual', 'model3']]
model_3.rename(columns={'model3' : 'predict'}, inplace=True)
pd.crosstab(model_3.actual, model_3.predict)

predict,cat,dog
actual,,
cat,893,853
dog,1599,1655


In [131]:
model_4 = cats_and_dogs_df[['actual', 'model4']]
model_4.rename(columns={'model4' : 'predict'}, inplace=True)
pd.crosstab(model_4.actual, model_4.predict)

predict,cat,dog
actual,,
cat,603,1143
dog,144,3110


<li>In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

In [132]:
# Define "dog" as baseline outcome
model_1['baseline'] = "dog"
model_1.head()

,actual,predict,baseline
0,cat,cat,dog
1,dog,dog,dog
2,dog,cat,dog
3,dog,dog,dog
4,cat,cat,dog


In [133]:
model_1_accuracy = (model_1.predict == model_1.actual).mean()
base_accuracy = (model_1.baseline == model_1.actual).mean()

print(f"model 1 accuracy:  {model_1_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 1 accuracy:  80.74%
baseline accuracy:  65.08%


In [134]:
model_2['baseline'] = "dog"
model_2_accuracy = (model_2.predict == model_2.actual).mean()
base_accuracy = (model_2.baseline == model_2.actual).mean()

print(f"model 2 accuracy:  {model_2_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 2 accuracy:  63.04%
baseline accuracy:  65.08%


In [135]:
model_3['baseline'] = "dog"
model_3_accuracy = (model_3.predict == model_3.actual).mean()
base_accuracy = (model_3.baseline == model_3.actual).mean()

print(f"model 3 accuracy:  {model_3_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 3 accuracy:  50.96%
baseline accuracy:  65.08%


In [136]:
model_4['baseline'] = "dog"
model_4_accuracy = (model_4.predict == model_4.actual).mean()
base_accuracy = (model_4.baseline == model_4.actual).mean()

print(f"model 4 accuracy:  {model_4_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 4 accuracy:  74.26%
baseline accuracy:  65.08%


#### Models 1 and 4 have higher accuracies higher than the baseline.

<li>Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recomend for Phase I? For Phase II?

In [137]:
# Since phase I is unsupervised, the cost of a false negative is high since that picture will not make it to phase II.
# The cost of false positives are low because a human can intervene in phase II,
# so between model 1 and 4 (high accuracy models), we should use recall to determine which to use in phase I.

model_1_subset = model_1[model_1.actual == 'dog']
model_1_subset.head()

,actual,predict,baseline
1,dog,dog,dog
2,dog,cat,dog
3,dog,dog,dog
5,dog,dog,dog
8,dog,dog,dog


In [138]:
model_1_recall = (model_1_subset.predict == model_1_subset.actual).mean()
base_recall = (model_1_subset.baseline == model_1_subset.actual).mean()

print(f"model 1 recall:  {model_1_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 1 recall:  80.33%
baseline recall:  100.00%


In [139]:
model_4_subset = model_4[model_4.actual == 'dog']
model_4_recall = (model_4_subset.predict == model_4_subset.actual).mean()
base_recall = (model_4_subset.baseline == model_4_subset.actual).mean()

print(f"model 4 recall:  {model_4_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 4 recall:  95.57%
baseline recall:  100.00%


### I would choose model 4 for phase I since it has a higher recall than model 1.

### For phase II, since human supervision is present, false negatives and false positives become less costly, so I would use model 1 since it is the more accurate model.

In [144]:
# Can also minimize false positives with precision.

<li>Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recomend for Phase I? For Phase II?

In [72]:
# Define cat as the baseline outcome
model_1['baseline'] = "cat"
model_1.head()

,actual,predict,baseline
0,cat,cat,cat
1,dog,dog,cat
2,dog,cat,cat
3,dog,dog,cat
4,cat,cat,cat


In [73]:
model_1_subset = model_1[model_1.actual == 'cat']
model_1_recall = (model_1_subset.predict == model_1_subset.actual).mean()
base_recall = (model_1_subset.baseline == model_1_subset.actual).mean()

print(f"model 1 recall:  {model_1_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 1 recall:  81.50%
baseline recall:  100.00%


In [74]:
model_2['baseline'] = "cat"
model_2_subset = model_2[model_2.actual == 'cat']
model_2_recall = (model_2_subset.predict == model_2_subset.actual).mean()
base_recall = (model_2_subset.baseline == model_2_subset.actual).mean()

print(f"model 2 recall:  {model_2_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 2 recall:  89.06%
baseline recall:  100.00%


In [75]:
model_3['baseline'] = "cat"
model_3_subset = model_3[model_3.actual == 'cat']
model_3_recall = (model_3_subset.predict == model_3_subset.actual).mean()
base_recall = (model_3_subset.baseline == model_3_subset.actual).mean()

print(f"model 3 recall:  {model_3_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 3 recall:  51.15%
baseline recall:  100.00%


In [76]:
model_4['baseline'] = "cat"
model_4_subset = model_4[model_4.actual == 'cat']
model_4_recall = (model_4_subset.predict == model_4_subset.actual).mean()
base_recall = (model_4_subset.baseline == model_4_subset.actual).mean()

print(f"model 4 recall:  {model_4_recall:.2%}")
print(f"baseline recall:  {base_recall:.2%}")

model 4 recall:  34.54%
baseline recall:  100.00%


### Choose model 2 for phase I cats

In [79]:
# Determine highest accuracy model
model_1_accuracy = (model_1.predict == model_1.actual).mean()
base_accuracy = (model_1.baseline == model_1.actual).mean()

print(f"model 1 accuracy:  {model_1_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 1 accuracy:  80.74%
baseline accuracy:  34.92%


In [80]:
model_2_accuracy = (model_2.predict == model_2.actual).mean()
base_accuracy = (model_2.baseline == model_2.actual).mean()

print(f"model 2 accuracy:  {model_2_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 2 accuracy:  63.04%
baseline accuracy:  34.92%


In [81]:
model_3_accuracy = (model_3.predict == model_3.actual).mean()
base_accuracy = (model_3.baseline == model_3.actual).mean()

print(f"model 3 accuracy:  {model_3_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 3 accuracy:  50.96%
baseline accuracy:  34.92%


In [82]:
model_4_accuracy = (model_4.predict == model_4.actual).mean()
base_accuracy = (model_4.baseline == model_4.actual).mean()

print(f"model 4 accuracy:  {model_4_accuracy:.2%}")
print(f"baseline accuracy:  {base_accuracy:.2%}")

model 4 accuracy:  74.26%
baseline accuracy:  34.92%


#### I would choose model 1 for phase II cats.

In [145]:
# Choose model 4 for highest precision in phase II cats

### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.

In [84]:
skl_acc_m1 = accuracy_score(model_1.actual, model_1.predict)
skl_acc_m1

0.8074

In [93]:
skl_prec_m1 = precision_score(model_1.actual, model_1.predict, pos_label='cat')
skl_prec_m1

0.6897721764420747

In [94]:
skl_recall_m1 = recall_score(model_1.actual, model_1.predict, pos_label='cat')
skl_recall_m1

0.8150057273768614

In [96]:
print(classification_report(model_1.actual, model_1.predict))

              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

